In [1]:
from tqdm import tqdm, trange
import json
from collections import namedtuple, defaultdict
import pandas as pd
import numpy as np
import torch
import os
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.models.word2vec import Word2Vec
import random
from scipy.sparse import csr_matrix
import dgl
import json
from torch_geometric.data import Data

import pickle as pkl
import sys
sys.path.append('/mnt/nfs/zhangtl/utils')
from util import myout

Using backend: pytorch


## load

In [2]:
dataset = 'aps'
df = pd.read_csv(f'../../raw_data/{dataset}/Papers.csv')
df.dropna(subset=['p_id', 'date'], inplace=True)
df['year'] = df['date'].apply(lambda u: int(u.split('-')[0]))
df.fillna({'refs': '', 'aids': '', 'v_id': '', 'kids': ''}, inplace=True)
df

/home/ztl/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,title,p_id,date,aids,v_id,kids,refs,year
0,A Study of the Transmission Spectra of Certain...,10.1103/PhysRevSeriesI.1.1,1893-07-01,302635,PRI,,,1893
1,New Books,10.1103/PhysRevSeriesI.1.66,1893-07-01,,PRI,,,1893
2,Note. Geometrical Proof of the Three-Ammeter M...,10.1103/PhysRevSeriesI.1.59,1893-07-01,"302641,302642",PRI,,,1893
3,Notes,10.1103/PhysRevSeriesI.1.62,1893-07-01,,PRI,,,1893
4,On The Infra-red Spectra of the Alkalies,10.1103/PhysRevSeriesI.1.28,1893-07-01,302655,PRI,,,1893
...,...,...,...,...,...,...,...,...
700030,Thermodynamic uncertainty relations for many-b...,10.1103/PhysRevE.104.064141,2021-12-30,"1283804,1283805",PRE,,"10.1103/PhysRevE.49.2726,10.1103/PhysRevLett.8...",2021
700031,Tolman-Ehrenfest-Klein law in non-Riemannian g...,10.1103/PhysRevD.104.124089,2021-12-30,"2008771,2008772",PRD,,"10.1103/PhysRev.35.904,10.1103/PhysRev.36.1791...",2021
700032,Transfer matrix method for light propagation i...,10.1103/PhysRevE.104.064702,2021-12-30,"1285151,1285152,1285153,1285154,1285155,128515...",PRE,,"10.1103/PhysRevLett.25.577,10.1103/PhysRevA.9....",2021
700033,Two-dimensional oxides assembled by <math xmln...,10.1103/PhysRevResearch.3.043231,2021-12-30,"306334,306335,306336,306337,306338",PRRESEARCH,"23,0,5","10.1103/PhysRev.140.A1133,10.1103/PhysRevLett....",2021


In [3]:
start_year, end_year = 2000, 2022
drop_id = df[(df.year < start_year) | (df.year >= end_year)].index
print(f'Drop {len(drop_id)} rows') # 691496
papers = df.drop(drop_id) # 1685097
papers

Drop 293905 rows


,title,p_id,date,aids,v_id,kids,refs,year
293905,<i>Ab initio</i> calculations of phonons in <s...,10.1103/PhysRevB.61.272,2000-01-01,"461571,461572,461573",PRB,,"10.1103/PhysRev.152.705,10.1103/PhysRev.158.43...",2000
293906,<i>Ab initio</i> simulations of compressed liq...,10.1103/PhysRevB.61.909,2000-01-01,"468932,468933,468934,468935",PRB,,"10.1103/PhysRevLett.55.2471,10.1103/PhysRevB.4...",2000
293907,<i>In situ</i> and interrupted-growth studies ...,10.1103/PhysRevE.61.607,2000-01-01,"1266530,1266531,1266532,1266533,1266534",PRE,,"10.1103/PhysRevB.41.1111,10.1103/PhysRevLett.6...",2000
293908,<i>In situ</i> observation of the ferroelectri...,10.1103/PhysRevB.61.203,2000-01-01,"463983,463984,463985,463986,463987",PRB,,"10.1103/PhysRev.144.662,10.1103/PhysRevB.57.16...",2000
293909,"<span class=""aps-inline-formula""><math xmlns=""...",10.1103/PhysRevB.61.1500,2000-01-01,"468371,468372,468373,468374,468375,468376,468377",PRB,,"10.1103/PhysRevB.25.4515,10.1103/PhysRevLett.7...",2000
...,...,...,...,...,...,...,...,...
700030,Thermodynamic uncertainty relations for many-b...,10.1103/PhysRevE.104.064141,2021-12-30,"1283804,1283805",PRE,,"10.1103/PhysRevE.49.2726,10.1103/PhysRevLett.8...",2021
700031,Tolman-Ehrenfest-Klein law in non-Riemannian g...,10.1103/PhysRevD.104.124089,2021-12-30,"2008771,2008772",PRD,,"10.1103/PhysRev.35.904,10.1103/PhysRev.36.1791...",2021
700032,Transfer matrix method for light propagation i...,10.1103/PhysRevE.104.064702,2021-12-30,"1285151,1285152,1285153,1285154,1285155,128515...",PRE,,"10.1103/PhysRevLett.25.577,10.1103/PhysRevA.9....",2021
700033,Two-dimensional oxides assembled by <math xmln...,10.1103/PhysRevResearch.3.043231,2021-12-30,"306334,306335,306336,306337,306338",PRRESEARCH,"23,0,5","10.1103/PhysRev.140.A1133,10.1103/PhysRevLett....",2021


## 验证是否只出现一次源节点，之后，才出现在目标节点

In [7]:
pid2year = {}
pid2cnt = {}
show_2times = []
early_ref = []
for ii in trange(len(papers)): # 先确定每个paper的出现年份
    pid = str(papers['p_id'].iloc[ii])
    year = int(papers['year'].iloc[ii])
    if pid not in pid2year:
        pid2year[pid] = year
        pid2cnt[pid] = 1
    else:
        pid2cnt[pid] += 1
        show_2times.append(pid)

for ii in trange(len(papers)):
    refs = papers['refs'].iloc[ii]
    year = int(papers['year'].iloc[ii])
    if len(refs)>0:
        rlst = refs.split(',')
        for ref in rlst:
            ref = str(ref)
            
            if ref in pid2year and pid2year[ref]>year: # 若tgt出现了，且它的年份>当前年份，则出错
                early_ref.append(ref)
myout(show_2times, early_ref)

100%|██████████| 406130/406130 [00:11<00:00, 35287.70it/s]

show_2times : len=0, []
early_ref : len=2, ['10.1103/PhysRevE.83.041906', '10.1103/PhysRevSTPER.10.020119']


## build graph

In [4]:
dataset = 'aps'
p_id2emb = pkl.load(open(f'../../../01_process/data_relabel/{dataset}/p_id2emb.pkl', 'rb'))
myout(p_id2emb)
def update_idx(idx, dic, cnt, feats, rel, feat_dim, no_emb):
    if idx not in dic:
        dic[idx] = cnt
        cnt += 1
        if rel == 0:
            try:
                feats.append(torch.from_numpy(p_id2emb[idx]).to(torch.float32))
            except:
                feats.append(torch.rand(feat_dim).to(torch.float32)-1)
                no_emb += 1
        elif rel == 1:
            feats.append(torch.rand(feat_dim).to(torch.float32)+1)
        elif rel == 2:
            feats.append(torch.rand(feat_dim).to(torch.float32)+2)
        elif rel == 3:
            feats.append(torch.rand(feat_dim).to(torch.float32))
    return dic, cnt, feats, no_emb

p_id2emb : len=700035, dict([10.1103/PhysRevSeriesI.1.1: [ 6.37484789e-02 -1.56797662e-01 -5.31356275e-01 -2.94902354e-01
  2.31120452e-01  3.06525081e-01 -1.64697647e-01 -1.35186583e-01
  2.44683966e-01  1.04386955e-01  2.70847946e-01 -6.16349354e-02
  3.17574173e-01 -8.29862654e-02  1.76895708e-01 -1.39169414e-02
  1.54908419e-01 -4.55742106e-02 -1.03417464e-01  1.78361490e-01
 -1.57583714e-01  1.14076614e-01 -2.87553906e-01  1.59453943e-01
  1.34424781e-02  1.54614896e-01  8.59246105e-02  8.55579302e-02
 -1.00014448e-01 -8.96584243e-02  4.31428775e-02 -2.55747199e-01
 -3.47675048e-02  2.10791662e-01 -1.00793615e-01  2.39948973e-01
  1.22995190e-01  1.72893703e-01  2.27474242e-01  1.66270465e-01
 -1.22651057e-02 -1.42175406e-01 -1.30287170e-01  1.24966584e-01
  6.04621395e-02  9.48932301e-03  6.88199252e-02  1.25183612e-01
  1.06229082e-01  1.87827960e-01  2.43721791e-02  1.71982035e-01
 -1.64385721e-01  1.04352988e-01  2.46298879e-01  8.15109089e-02
 -1.89954434e-02  8.51822197e-02 

In [5]:
start_year, end_year = 2000, 2022
feat_dim = 128

id2nid, cnt, no_emb = {}, 0, 0
lst, feats = [], []
for i in trange(len(papers)):
    ts = int(papers['year'].iloc[i])
    p_id = str(papers['p_id'].iloc[i])
    refs = papers['refs'].iloc[i]
    
    id2nid, cnt, feats, no_emb = update_idx(p_id, id2nid, cnt, feats, 0, feat_dim, no_emb)
    if len(refs)>0:
        rlst = refs.split(',')
        for ref in rlst:
            ref = str(ref)
            if ref != '':
                id2nid, cnt, feats, no_emb = update_idx(ref, id2nid, cnt, feats, 0, feat_dim, no_emb)
                lst.append((id2nid[p_id], id2nid[ref], 0, ts))
        
feat = torch.stack(feats)
src = torch.tensor([item[0] for item in lst])
tgt = torch.tensor([item[1] for item in lst])
rel = torch.tensor([item[2] for item in lst])
tsp = torch.tensor([item[3] for item in lst])
myout(feat, src, tgt, rel, tsp)

100%|██████████| 406130/406130 [00:21<00:00, 19289.89it/s]


feat : shape=torch.Size([577852, 128])
tensor([[-0.0452, -0.4180, -0.1381,  ..., -0.3730,  0.0763,  0.4313],
        [-0.1554, -0.0979, -0.2336,  ..., -0.1276,  0.1056,  0.3306],
        [-0.0480, -0.4074, -0.1778,  ..., -0.3000,  0.0955,  0.4711],
        ...,
        [-0.0355, -0.1025,  0.0216,  ..., -0.2168,  0.0292,  0.0406],
        [-0.0477, -0.3570, -0.1027,  ..., -0.3685,  0.0045,  0.4755],
        [-0.0142, -0.4941, -0.2021,  ..., -0.3956,  0.0186,  0.4762]])
src : shape=torch.Size([6795779]), tensor([     0,      0,      0,  ..., 577851, 577851, 577851])
tgt : shape=torch.Size([6795779]), tensor([     1,      2,      3,  ..., 540503, 540793, 547191])
rel : shape=torch.Size([6795779]), tensor([0, 0, 0,  ..., 0, 0, 0])
tsp : shape=torch.Size([6795779]), tensor([2000, 2000, 2000,  ..., 2021, 2021, 2021])


In [7]:
graph = dgl.graph((src, tgt))
graph.ndata['feat'] = feat

nid2id = {vv: kk for kk, vv in id2nid.items()}
graph.ndata['raw_id'] = [nid2id[nid] for nid in range(0, len(feat))]

graph.edata['rel'] = rel
graph.edata['ts'] = tsp
graph

AttributeError: 'list' object has no attribute 'shape'

## build graph

In [9]:
inputs = pkl.load(open('../../../01_process/data_papers/aps/inputs.pkl', 'rb'))
inputs.keys()

dict_keys(['feat', 'eind', 'rel', 'id2nid', 'index'])

In [13]:
tsp = []
for kk, vv in inputs['index'].items():
    left, right = vv[1:]
    tsp.extend([kk]*(right-left))
tsp = torch.tensor(tsp)
myout(tsp)

tsp : shape=torch.Size([6795779]), tensor([2000, 2000, 2000,  ..., 2021, 2021, 2021])


In [39]:
graph = dgl.graph((inputs['eind'][0, :], inputs['eind'][1, :]))
graph.ndata['feat'] = inputs['feat']

graph.ndata['raw_nid'] = torch.arange(len(inputs['feat']))

graph.edata['rel'] = inputs['rel']
graph.edata['ts'] = tsp
graph

Graph(num_nodes=577852, num_edges=6795779,
      ndata_schemes={'feat': Scheme(shape=(128,), dtype=torch.float32), 'raw_nid': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'rel': Scheme(shape=(), dtype=torch.int64), 'ts': Scheme(shape=(), dtype=torch.int64)})

In [18]:
json.dump(inputs['id2nid'], open('../data/aps/id2nid.json', 'w'))

In [40]:
dgl.save_graphs('../data/aps/graph.bin', [graph])

## choose year

In [32]:
train_start, train_end = 2001, 2010
valid_start, valid_end = 2010, 2012

ts_eids = graph.filter_edges(lambda x: (x.data['ts']>=train_start) & (x.data['ts']<valid_end))
ts_graph = dgl.edge_subgraph(graph, ts_eids)
ts_graph

Graph(num_nodes=351251, num_edges=2709363,
      ndata_schemes={'feat': Scheme(shape=(128,), dtype=torch.float32), 'raw_nid': Scheme(shape=(), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'rel': Scheme(shape=(), dtype=torch.int64), 'ts': Scheme(shape=(), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)})

## two cites as valid test

In [35]:
valid_eids = ts_graph.filter_edges(lambda x: x.data['ts'] >= valid_start)
valid_src = ts_graph.edges()[0][valid_eids].numpy()
valid_tgt = ts_graph.edges()[1][valid_eids].numpy()
valid_tsp = ts_graph.edata['ts'][valid_eids].numpy()
myout(valid_src, valid_tgt, valid_tsp)

valid_src : shape=(612756,), [150004 150004 150004 ... 187778 187778 187778]
valid_tgt : shape=(612756,), [229807 214862   5930 ... 177679 177746 180749]
valid_tsp : shape=(612756,), [2010 2010 2010 ... 2011 2011 2011]


In [44]:
assert np.all(np.sort(valid_src) == valid_src)
pos_cuts = [0] + [pos for pos in range(len(valid_src)) if pos and valid_src[pos-1] != valid_src[pos]] + [len(valid_src)]
myout(pos_cuts)

pos_cuts : len=37776, list([0, 4, 7, ..., 612695, 612718, 612756])


In [47]:
val_test_src, val_tgt, test_tgt,  = [], [], []
val_ts, rest_pos = [], []
for ii in trange(len(pos_cuts)-1, desc='two_cites_as_valid_test'):
    left, right = pos_cuts[ii], pos_cuts[ii+1]
    if right - left < 3:
        continue
    
    val_test_src.append(valid_src[left])
    val_tgt.append(valid_tgt[left])
    test_tgt.append(valid_tgt[left+1])
    val_ts.append(valid_tsp[left])
    
    rest_pos.extend([pos for pos in range(left+2, right)])
myout(val_test_src, val_tgt, test_tgt, val_ts)

two_cites_as_valid_test: 100%|██████████| 37775/37775 [00:00<00:00, 298521.21it/s]

val_test_src : len=36265, list([150004, 150005, 150006, ..., 187776, 187777, 187778])
val_tgt : len=36265, list([229807, 269707, 190980, ..., 204062, 216300, 225487])
test_tgt : len=36265, list([214862, 19131, 193490, ..., 111381, 221682, 190537])
val_ts : len=36265, list([2010, 2010, 2010, ..., 2011, 2011, 2011])


In [53]:
rest_tsp = valid_tsp[rest_pos]
myout(rest_tsp)

rest_tsp : shape=(537843,), [2010 2010 2010 ... 2011 2011 2011]


In [48]:
_, ts_cuts = np.unique(np.array(val_ts), return_index=True)
ts_cuts = list(ts_cuts) + [len(val_ts)]
myout(ts_cuts)

ts_cuts : len=3, [0, 17962, 36265]


In [52]:
val_test_df = pd.DataFrame({'src': val_test_src, 'val_tgt': val_tgt, 'test_tgt': test_tgt, 'ts': val_ts})
val_test_df

,src,val_tgt,test_tgt,ts
0,150004,229807,214862,2010
1,150005,269707,19131,2010
2,150006,190980,193490,2010
3,150007,259276,344164,2010
4,150008,200091,188481,2010
...,...,...,...,...
36260,187774,255556,212191,2011
36261,187775,187952,289630,2011
36262,187776,204062,111381,2011
36263,187777,216300,221682,2011


## build train valid graph

In [50]:
rest_eind = torch.stack([torch.tensor(valid_src[rest_pos]), torch.tensor(valid_tgt[rest_pos])]) # [2, e1]

In [55]:
train_eids = ts_graph.filter_edges(lambda x: x.data['ts'] < train_end)
train_eind = torch.stack(ts_graph.edges())[:, train_eids] # [2, e2]
train_tsp = ts_graph.edata['ts'][train_eids]
myout(train_eind, train_tsp)

train_eind : shape=torch.Size([2, 2096607])
tensor([[     0,      0,      0,  ..., 150003, 150003, 150003],
        [187779, 187780, 187781,  ..., 112910, 117283, 130341]])
train_tsp : shape=torch.Size([2096607]), tensor([2001, 2001, 2001,  ..., 2009, 2009, 2009])


In [60]:
valid_eind = torch.hstack([train_eind, rest_eind]) # [2, e1+e2]
valid_tsp = torch.hstack([train_tsp, torch.tensor(rest_tsp)])
myout(valid_eind, valid_tsp)

valid_eind : shape=torch.Size([2, 2634450])
tensor([[     0,      0,      0,  ..., 187778, 187778, 187778],
        [187779, 187780, 187781,  ..., 177679, 177746, 180749]])
valid_tsp : shape=torch.Size([2634450]), tensor([2001, 2001, 2001,  ..., 2011, 2011, 2011])


In [76]:
def build_geo_graph(ts_graph, eind, tsp):
    num_nodes = eind.max() + 1
    x = ts_graph.ndata['feat'][0:num_nodes, :]
    raw_nid = ts_graph.ndata['raw_nid'][0:num_nodes]
    
    _, ts_cuts = np.unique(tsp.numpy(), return_index=True)
    ts_cuts = list(ts_cuts) + [len(tsp.numpy())]
    return Data(num_nodes=int(num_nodes), edge_index = eind, x=x, edge_attr=tsp, ts_cuts=ts_cuts, \
        raw_nid=raw_nid)
train_graph = build_geo_graph(ts_graph, train_eind, train_tsp)
valid_graph = build_geo_graph(ts_graph, valid_eind, valid_tsp)
myout(train_graph, valid_graph)

train_graph = Data(x=[344164, 128], edge_index=[2, 2096607], edge_attr=[2096607], num_nodes=344164, ts_cuts=[10], raw_nid=[344164])
valid_graph = Data(x=[351251, 128], edge_index=[2, 2634450], edge_attr=[2634450], num_nodes=351251, ts_cuts=[12], raw_nid=[351251])


## gen index

In [65]:
_, ts_cuts = np.unique(train_graph.edge_attr.numpy(), return_index=True)
ts_cuts = list(ts_cuts) + [len(train_graph.edge_attr.numpy())]
myout(ts_cuts)

ts_cuts : len=10, [0, 180547, 375878, 565779, 780167, 1025608, 1275215, 1529317, 1812759, 2096607]


In [66]:
_, ts_cuts = np.unique(valid_graph.edge_attr.numpy(), return_index=True)
ts_cuts = list(ts_cuts) + [len(valid_graph.edge_attr.numpy())]
myout(ts_cuts)

ts_cuts : len=12, list([0, 180547, 375878, ..., 2096607, 2361546, 2634450])


In [70]:
train_graph.num_nodes, train_graph.ts_cuts

(344164,
 [0,
  180547,
  375878,
  565779,
  780167,
  1025608,
  1275215,
  1529317,
  1812759,
  2096607])

## labels

In [72]:
data_path = f'../../../01_process/data_papers/aps/'
labels_cum_log = pkl.load(open(os.path.join(data_path, 'labels_cum_log.pkl'), 'rb'))
labels_cum_log[2001]

,id,nid,2002,2003,2004,2005,2006,2007,2008,2009,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,10.1103/PhysRevLett.86.26,83711,1.098612,1.609438,1.791759,1.791759,1.791759,1.791759,1.791759,1.791759,...,1.791759,1.791759,1.791759,1.791759,1.791759,2.079442,2.197225,2.197225,2.197225,2.197225
1,10.1103/PhysRevLett.86.22,83713,1.609438,1.945910,2.197225,2.397895,2.564949,2.639057,2.772589,2.890372,...,3.258096,3.332205,3.583519,3.688879,3.737670,3.850148,3.970292,4.189655,4.248495,4.276666
2,10.1103/PhysRevLett.86.115,83715,1.386294,1.386294,1.609438,1.945910,1.945910,1.945910,1.945910,1.945910,...,1.945910,1.945910,1.945910,1.945910,1.945910,1.945910,2.079442,2.079442,2.079442,2.079442
3,10.1103/PhysRevLett.86.179,83716,0.000000,0.693147,0.693147,0.693147,1.098612,1.609438,1.791759,1.945910,...,2.302585,2.302585,2.302585,2.302585,2.397895,2.397895,2.397895,2.397895,2.397895,2.397895
4,10.1103/PhysRevLett.86.63,83721,2.197225,2.564949,2.639057,2.772589,2.995732,2.995732,3.178054,3.218876,...,3.258096,3.332205,3.401197,3.465736,3.610918,3.688879,3.688879,3.737670,3.761200,3.806663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14771,10.1103/PhysRevB.65.045102,122891,0.000000,0.000000,0.693147,1.098612,1.098612,1.098612,1.386294,1.386294,...,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.609438,1.791759,2.079442
14772,10.1103/PhysRevB.65.033312,122895,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
14773,10.1103/PhysRevB.65.045307,122898,0.000000,0.000000,0.693147,1.609438,1.609438,1.609438,1.609438,1.609438,...,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438
14774,10.1103/PhysRevB.65.035213,122899,0.000000,0.000000,0.693147,1.098612,1.098612,1.098612,1.098612,1.098612,...,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294


In [77]:
labels_cum_log[2000]

,id,nid,2001,2002,2003,2004,2005,2006,2007,2008,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,10.1103/PhysRevB.61.272,0,1.098612,1.386294,1.386294,1.609438,1.609438,1.609438,1.609438,1.945910,...,2.302585,2.302585,2.484907,2.639057,2.639057,2.639057,2.708050,2.708050,2.772589,2.944439
1,10.1103/PhysRevB.61.909,18,2.302585,2.484907,2.708050,2.995732,3.091043,3.091043,3.178054,3.218876,...,3.433987,3.433987,3.433987,3.433987,3.496508,3.496508,3.496508,3.496508,3.496508,3.496508
2,10.1103/PhysRevB.61.1,35,2.397895,2.772589,2.995732,3.135494,3.178054,3.178054,3.258096,3.367296,...,3.737670,3.784190,3.806663,3.850148,3.891820,3.931826,3.931826,3.931826,3.951244,3.951244
3,10.1103/PhysRevE.61.607,36,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
4,10.1103/PhysRevB.61.203,40,1.098612,1.791759,1.791759,1.791759,1.791759,1.791759,1.791759,1.791759,...,2.079442,2.079442,2.079442,2.079442,2.079442,2.079442,2.079442,2.079442,2.079442,2.079442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14453,10.1103/PhysRevB.63.060402,83701,0.693147,1.609438,1.945910,2.079442,2.197225,2.302585,2.484907,2.639057,...,2.639057,2.639057,2.708050,2.708050,2.708050,2.708050,2.708050,2.708050,2.708050,2.708050
14454,10.1103/PhysRevD.63.035001,83703,0.693147,1.791759,2.079442,2.302585,2.484907,2.639057,2.708050,2.995732,...,3.044523,3.135494,3.135494,3.178054,3.178054,3.218876,3.258096,3.295837,3.295837,3.332205
14455,10.1103/PhysRevB.63.033305,83706,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
14456,10.1103/PhysRevD.63.034503,83709,2.302585,2.639057,2.833213,3.044523,3.258096,3.295837,3.332205,3.401197,...,3.465736,3.465736,3.496508,3.496508,3.526361,3.526361,3.526361,3.526361,3.526361,3.526361


In [74]:
myout(ts_graph.ndata['raw_nid'])

_73 : shape=torch.Size([351251]), tensor([ 83711,  83713,  83715,  ..., 363117, 334220, 348543])
